## Running prep_chem_src
- based on part 1 prep_chem_SRC was compiled and ready to executed for emission inventory creation. 
- Following WRF-chem nepal tutorial and PREP_CHEM_SRC, README edited ```prep_chem_sources.inp```.
- then run the preogram by executing ```./prep_chem_sources_RADM_WRF_FIM.exe```, it ran for some steps but exited with error of ```Warning! ***HDF5 library version mismatched error***``` saying the HDF5 version used in compiling is 1.8.8 and version for running PREP_CHEM_SRC is 1.8.12 
- to check what hdf5 is used by prep_che_src, run a command ``` h5dump -V``` which gives ```h5dump: Version 1.8.12```, so there is a problem. 
- as per the long well written error, 'LD_LIBRARY_PATH' has to be checked, based  on [this](http://askubuntu.com/questions/9878/set-ld-library-path-for-applications-started-from-the-desktop) and [this](http://askubuntu.com/questions/9878/set-ld-library-path-for-applications-started-from-the-desktop), a temproray solution is to execute ```export LD_LIBRARY_PATH="/path/to/sdk/lib"```, in this case ```export LD_LIBRARY_PATH="/path_libs/hdf5-1.8.8"```, but no error correction is happend, it was checked by executing the command ``` h5dump -V``` but showing the same version and so problem persists.
- followed several other options in editing the ld library path but faced dead bloack in terms of removing a entry in the ldconfig, it can be viewed by giving ```ldconfig -V``` a super note on [this](http://www.cyberciti.biz/tips/linux-shared-library-management.html) but can't edit, so option is to compile hdf5 in that sepcifid location istead of changing the path so chnaged the compileed folder name and new foldeer is unziped from hdf5 source and execute this
```python
FC=gfortran CC=gcc CXX=gc++ ./configure  --prefix=/usr/local --with-zlib=/path_libs/zlib-1.2.5  --disable-shared --enable-fortran
```
then ```sudo make``` and ```sudo make check``` and ```sudo make checkinstall```, given installation successful, the check was made by running ```h5dump -V``` this time it gives what wanted that is version 1.8.8 and rerun the prep_chem_src, but still giving the same error, but now got a clue that no shared libarryies are over written above the old one version of 1.8.12, so the clue was first configure command was executed with ```disable-shared``` making it enable would solve the problem. 
- so redoing it by uninstalling, hdf5 by ```sudo apt-get remove hdf5``` and reconfiguring by ```FC=gfortran CC=gcc CXX=gc++ ./configure  --prefix=/usr/local --with-zlib=/path_libs/zlib-1.2.5  --enable-shared --enable-fortran``` and following athoer comments like, ```sudo make```, ```sudo make check``` and ```sudo checkinstall```. But now running ```h5dump -v``` gives error of ```h5dump: error while loading shared libraries: libhdf5.so.7: cannot open shared object file: No such file or directory```. so execcuted ldconfig as per [this](http://www.cyberciti.biz/tips/linux-shared-library-management.html), ```ldconfig -l /usr/local/lib/libhdf5.so.7.0.2``` and again executing ```ldconfig```, now ```h5dump -v``` gives version wanted 1.8.8 without any error. this time also version mismatch error is giving, so decided to removed any shared libaryr as from version 1.8.12, so removed files such as from /usr/local/lib, some four files ending with libhdf5.so.8. still the problem persist with prep_chem_src saying ```libhdf5.so.8: cannot open shared object file: No such file or directory
```. so the final soultion would be to recompile the prep_chem_src with same shared libary hdf and main hdf5 in same version.
- So copying include file from old compile, and executing this ```make OPT=opt.gfortran CHEM=RADM_WRF_FIM``` made prep_che_src and so copied the olde input file and executed the program, but failed in opeining nc file, dump error.
- so again made ```ldd ./prep_chem_sources_RADM_WRF_FIM.exe```, and it gives two lines fo rntecdf files such as this, libnetcdff.so.5 => /usr/lib/libnetcdff.so.5 (0x00007f8eedbb9000) and libnetcdf.so.6 => /usr/lib/libnetcdf.so.6 (0x00007f8eeca4c000), but chcking the file location and iclude file netcdf path, there is adescripiancy, so changed the file ```include.mk.opt.gfortran``` and it is as follows
```bash
#Makefile include include.mk.opt
############################## Change Log ##################################
# 1.0.0.2
#
# 000908 MJB include.mk-mrc ##
#            Added MAKE environment varaible.
#            Added free format option to F_OPTS for some platforms. ##
# 000907 MJB include.mk-mrc ##
#            Changed the defualts to no NCAR Graphics and no parallel.
#            Also commented out the machine specifics to force the user to
#            select the appropriate machine for them. ##
# 000823 MJB include.mk-mrc ##
#            New - defines all make environment varaibles and is included
#            in all make files. ##
#
############################################################################
# Define make (gnu make works best).
MAKE=/usr/bin/make
# Activate appropriate parts below, comment out others.
# NCAR Graphics.
#---------------------------------------------------------------
# If you are using a standard installation of NCAR Graphics, set:
#       LOADER=ncargf90
# in the machine-dependent sections below
#LIBNCARG=
#---------------------------------------------------------------
# If you are using the NCAR dummy libraries...
NCARG_DIR=$(BASE)
#LIBNCARG=-L$(NCARG_DIR) -lncarg-$(UTILS_VERSION) -lncarg_c-$(UTILS_VERSION) \
#                        -lncarg_gks-$(UTILS_VERSION)
#LIBNCARG=-L$(NCARG_DIR) -lncarg-$(UTILS_VERSION) 
LIBNCARG=$(BASE)/libncarg-$(UTILS_VERSION)-$(OPT).a
#---------------------------------------------------------------
# If you are using a real distribution of NCAR Graphics...
#NCARG_DIR=/usr/local/ncarg-4.3.0/lib
#LIBNCARG=-L$(NCARG_DIR) -lncarg -lncarg_gks -lncarg_c -L/usr/X11R6/lib -lX11 -ldl
#---------------------------------------------------------------
# NETCDF libraries
NETCDF=/usr
NETCDF_INC=-I$(NETCDF)/include
NETCDF_LIBS=-L$(NETCDF)/lib -lnetcdf -lnetcdff
# HDF libraries
HDF5=/usr/local
HDF5_INC=-I$(HDF5)/include
HDF5_LIB=-L$(HDF5)/lib -lhdf5hl_fortran -lhdf5_fortran -lhdf5_hl -lhdf5 -L/path-zlib-1.2.5/lib/ -l
# Machine-dependent options.
#-----------------  LINUX Portland Group pgf77/gcc ---------------
CMACH=PC_LINUX1
F_COMP=gfortran
C_COMP=gcc
LOADER=gfortran
C_LOADER=gcc
LIBS=
MOD_EXT=mod
F_OPTS=  -Xpreprocessor -D$(CHEM) -O2 -frecord-marker=4
C_OPTS= -O2 #-DUNDERSCORE -DLITTLE
LOADER_OPTS= -O2
#-----------------------------------------------------------------
# If compiling for a single-CPU platform only (without MPI):
#-----------------------------------------------------------------
PAR_LIBS=
PAR_DEFS=
#-----------------------------------------------------------------
# For IBM,HP,SGI,ALPHA use these:
ARCHIVE=ar rs
# For NEC SX-6
#ARCHIVE=sxar rs
# For SUN,CONVEX
#ARCHIVE=ar r'
```
- now again compiled and this time prep_chem_src run without any error upto no such file or folder error for some specific emission options so, edited that and rerun, it run without error and made bin file namely ```FIRE-RRCH-T-2014-03-01-000000-g1-ab.bin, FIRE-RRCH-T-2014-03-01-000000-g1-bb.bin, FIRE-RRCH-T-2014-03-01-000000-g1-gocartBG.bin, the final input ```prep_chem_sources.inp``` file is as follows
```bash
$RP_INPUT
!##########################################################################!
!  CATT-BRAMS/MCGA-CPTEC emission model    CPTEC/INPE                      !
!  version 1: 28/feb/2007                                                  !
!  developed/coded by Saulo Freitas and Karla Longo                        !
!  contact: sfreitas@cptec.inpe.br   - www.cptec.inpe.br/meio_ambiente     !
!###########################################################################
!---------------- grid_type of the grid output
   grid_type= 'lambert',	
   		       !'rams' = rams grid 
   		       ! 'polar'  = polar sterog. grid output
                       ! 'gg'     = gaussian grid output
		       ! 'll'     = lat/lon grid output
		       ! 'lambert'  = lambert grid output
		       ! 'mercator' = mercator grid output
!---------------- if the output data is for use in CATT-BRAMS model, provide at least one analysis file 
!---------------- of this model
   !rams_anal_prefix = './ANL/barca',
   !rams_anal_prefix = '/dados/dados2/mfapel/TESTCASE/ANL/A',
!---------------- date of emission  
!---------------- date of emission
!   ihour=00, iday=2, imon=10, iyear=2008,
    ihour=00, 
    iday=01, 
    imon=03, 
    iyear=2014,
!---------------- select the sources datasets to be used   
   use_retro =1,  ! 1 = yes, 0 = not
   retro_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/RETRO/anthro',
   use_edgar =2,  ! 0 - not, 1 - Version 3, 2 - Version 4 for some species
   edgar_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/EDGARV4',
   use_gocart =1,
   gocart_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/GOCART/emissions',
   use_fwbawb =0,
   fwbawb_data_dir ='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/Emissions_Yevich_Logan',
   use_bioge =0, ! 1 - geia, 2 - megan 
   ! ###### 
   ! # BIOGENIC = 1
   !bioge_data_dir ='/home/poluicao/EMISSION_DATA/biogenic_emissions',
   ! # MEGAN = 2
   ! ######   
   bioge_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/biogenic_emissions',   
   ! ######
   use_gfedv2 =0,
   gfedv2_data_dir ='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/GFEDv2-8days',
   use_bbem =0,
   use_bbem_plumerise =0,
!---------------- if  the merging of gfedv2 with bbem is desired (=1, yes, 0 = no)  
   merge_GFEDv2_bbem =0,
!---------------- Fire product for BBBEM/BBBEM-plumerise emission models
!  bbem_wfabba_data_dir   ='/lfs0/projects/wrf-chem/wrfchemv33/Emission_data/fires_data/WF_ABBA_v60/filt/f',
!   bbem_wfabba_data_dir   ='/public/data/sat/ssec/goes11/wf_abba/f',
!  bbem_wfabba_data_dir   ='/lfs0/projects/wrf-chem/wrfchemv33/WFABBA/f',
!  bbem_wfabba_data_dir   ='/lfs0/projects/wrf-chem/wrfchemv33/WFABBA/f',
!  bbem_modis_data_dir    ='/lfs0/projects/wrf-chem/wrfchemv33/Emission_data/fires_data/MODIS/Fires.',
!   bbem_modis_data_dir    ='/public/data/sat/firms/global/Global_MCD14DL_',
!   bbem_inpe_data_dir     ='/lfs0/projects/wrf-chem/wrfchemv33/Emission_data/fires_data/DSA/Focos',
!   bbem_extra_data_dir    ='/lfs0/projects/wrf-chem/wrfchemv33/Emission_data/fires_data/BLMALASKA/current.dat',
!---------------- veg type data set (dir + prefix)
! veg_type_data_dir      ='/home/poluicao/SURFACE_DATA/GL_IGBP_AVHRR_INPE/IGBP',	       
  veg_type_data_dir      ='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/surface_data/GL_IGBP_MODIS_INPE',	         
!---------------- vcf type data set (dir + prefix)
  use_vcf = 0,
  vcf_type_data_dir      ='/dados/dados3/stockler/VCF/data_out/2004/VCF',	       
!----------------  Carbon density data  ----------------
!----------------  New Olson''s data set (dir + prefix)
!  olson_data_dir= '/home/poluicao/EMISSION_DATA/OLSON2/OLSON',   
   olson_data_dir= '/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/OLSON2/OLSON',      
!----------------  Old Olson''s data set
  carbon_density_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/surface_data/GL_OGE_INPE/OGE', 
!---------------- carbon density data set for Amazon (dir + prefix)
!  fuel_data_dir      =' ',		     
!---------------- gocart background
  use_gocart_bg =1,
  gocart_bg_data_dir='/home/swl-sacon-dst/Documents/GISE_2013/LAB/WRF-chem/wrf_chem_tut/Global_emissions_v3/Emission_data/GOCART/',
!---------------- volcanoes emissions
!   use_volcanoes =1,
!   volcano_index =1143, !Popocatepetl
   use_these_values='NONE',
! define a text file for using external values for INJ_HEIGHT, DURATION,
! MASS ASH (units are meters - seconds - kilograms) and the format for
! a file 'values.txt' is like this:
! 11000. 10800. 1.5e10
! use_these_values='values.txt', 
!  begin_eruption='201204200000',  !begin time UTC of eruption YYYYMMDDhhmm   
!   begin_eruption='201305080000',  !begin time UTC of eruption YYYYMMDDhhmm   
!---------------- degassing volcanoes emissions
!   use_degass_volcanoes =0,
!   degass_volc_data_dir ='/home/poluicao/EMISSION_DATA/VOLC_SO2', 
!---------------- user specific  emissions directory
!---------------- Update for South America megacities
!---------------- set 'NONE' if you do not want to use
!  user_data_dir='/home/poluicao/EMISSION_DATA/SouthAmerica_Megacities',
   user_data_dir='NONE',
!--------------------------------------------------------------------------------------------------
   pond=1,   ! mad/mfa  0 -> molar mass weighted 
             !          1 -> Reactivity weighted   
!---------------- for grid type 'll' or 'gg' only
   grid_resolucao_lon=0.2,
   grid_resolucao_lat=0.2,
   nlat=320,          ! if gg (only global grid)
   lon_beg   = -180., ! (-180.:+180.) long-begin of the output file
   lat_beg   = -90., ! ( -90.:+90. ) lat -begin of the output file
   delta_lon = 360., ! total long extension of the domain (360 for global)
   delta_lat = 180., ! total lat  extension of the domain (180 for global)
!---------------- For regional grids (polar or lambert)
   NGRIDS   = 1,            ! Number of grids to run
   NNXP     = 90 !,201,86,46,        ! Number of x gridpoints
   NNYP     = 90 ! ,311,74,46,        ! Number of y gridpoints
   NXTNEST  = 0  !,1,2,3,          ! Grid number which is the next coarser grid
   DELTAX   = 100000.
   DELTAY   = 100000.  ! X and Y grid spacing
   ! Nest ratios between this grid and the next coarser grid.
   NSTRATX  = 1,2,3,4,           ! x-direction
   NSTRATY  = 1,2,3,4,           ! y-direction
   NINEST = 1,26,22,22,        ! Grid point on the next coarser
   NJNEST = 1,7,15,32,        !  nest where the lower southwest
                             !  corner of this nest will start.
                             !  If NINEST or NJNEST = 0, use CENTLAT/LON
   POLELAT  = 18.363,           ! If polar, latitude/longitude of pole point
   POLELON  = 78.925,       ! If lambert/mercator, lat/lon of grid origin (x=y=0.) (ref_lat
                             ! /ref_lon from namelist.wps)
   STDLAT1  = 18.363,           ! If polar, unused
   STDLAT2  = 18.363,           ! If lambert, standard latitudes of projection 
                             !(truelat2/truelat1 from namelist.wps, STDLAT1 < STDLAT2)
                             ! If mercator STDLAT1 = 1st true latitude 
   CENTLAT  = 18.363,      !45.,  -23., 27.5,  27.5,   ! (ref_lat/ref_lon from namelist.wps)
   CENTLON  = 78.925,      !		-99.18, -46.,-80.5, -80.5,
!---------------- model output domain for each grid (only set up for rams)
   lati =  -90.,  -90.,   -90., 
   latf =  +90.,  +90.,   +90.,  
   loni = -180., -180.,  -180., 
   lonf =  180.,  180.,   180., 
!---------------- project rams grid (polar sterogr) to lat/lon: 'YES' or 'NOT'
   proj_to_ll='NO', 
!---------------- output file prefix (may include directory other than the current)
   chem_out_prefix = 'FIRE-RRCH',
   chem_out_format = 'vfm',
!---------------- convert to WRF/CHEM (yes,not)
  special_output_to_wrf = 'YES',
  $END
```
- with this prep_chem_src was made and then executed the program with input file, some misatakes in emission source non availablity was raised, it was solved in input file and then re run the program, it completed without any error and made three files in the bin folder ```FIRE-RRCH-T-2014-03-01-000000-g1-ab.bin```, ```FIRE-RRCH-T-2014-03-01-000000-g1-bb.bin``` and ```FIRE-RRCH-T-2014-03-01-000000-g1-gocartBG.bin```. three totally cost arround only 7 MB in size!